### Standard setup for the demo

In [1]:
import sys
sys.path.append(r'\absolute\path\of\ni-streamer\py_api')
from nistreamer import NIStreamer

In [2]:
ni_strmr = NIStreamer()

ao_card = ni_strmr.add_ao_card('Dev2', samp_rate=400e3)
do_card = ni_strmr.add_do_card('Dev3', samp_rate=10e6)

ao_0 = ao_card.add_chan(chan_idx=0)
do_0 = do_card.add_chan(port_idx=0, line_idx=0)

In [3]:
START_TRIG = 'PFI0'
ao_card.start_trig_out = START_TRIG
do_card.start_trig_in = START_TRIG
ni_strmr.starts_last = ao_card.max_name

In [4]:
# Minimal demo sequence
ni_strmr.clear_edit_cache()
ao_0.sine(t=0, dur=100e-3, amp=1.0, freq=12.34)
do_0.high(t=50e-3, dur=150e-3)
ni_strmr.compile()

0.2

# Context manager stream interface

There are 2 types of stream control interfaces:
- basic built-in `run` method;
- context manager API.

A minimal example of context manager usage:

In [5]:
with ni_strmr.init_stream() as stream_handle:
    stream_handle.launch()
    stream_handle.wait_until_finished()

Breakdown:

* `with ni_strmr.init_stream() as stream` initializes the stream and assigns `StreamHandle` instance to `stream` target;

* `launch()` method commands to launch the run. **Note** - this call is non-blocking and returns immediately;

* `wait_until_finished()` blocks and returns only when the full waveform generation is finished;

* Stream is automatically closed and all resources are released when leaving the context due to any reason.

<div class="alert alert-block alert-info"> 
    <b>NOTE</b> You should always make a `wait_until_finished` call every time after you called `launch` even if you are sure the generation has finished already. This call is necessary to toggle the internal state machine from `Running` back to `Idle` state.
</div>

## Context manager use cases

1. Adding custom logic to run before/during/after each repetition without stream re-init overhead - NI tasks, worker threads, and large memory buffers are only allocated once when entering the context and are re-used for each launch as long as you are staying within the context block;
   
1. Using the in-stream looping feature.

### (1) A series of launches without re-init overhead

Example applications:
* Wait for start trigger for each run;
* Run custom code before/after each run (e.g. print progress, send network commands to other instruments, save data).

In [6]:
nreps = 100
with ni_strmr.init_stream() as stream:
    for rep_idx in range(nreps):
        # before-run custom code here
        stream.launch()  # if start trigger was configured, each launch will wait for a trig to start
        stream.wait_until_finished()
        # after-run custom code here
        print(f'{rep_idx + 1} reps finished out of {nreps}', end='\r')

(_Note:_ this is how the legacy `run(nreps)` method is implemented under the hood)

### (2) In-stream looping feature

In the example above we repeated the sequence by **re-launching** the stream several times.

`StreamHandle` exposes another way - **in-stream looping** feature. A minimal example:

In [7]:
with ni_strmr.init_stream() as stream:
    stream.launch(instream_reps=100)
    stream.wait_until_finished()

**In-stream looping** is very different from **repetitive re-launching**:

- For in-stream looping, the stream is started once and generation goes over the sequence `instream_reps` times "on-the-fly" as if they were concatenated together. This means there will be a minimal gap between subsequent repetitions which will not fluctuate. But it will only await for a single start trigger in the beginning if `start_trig` was configured.

- For repetitive re-launching, the stream is stopped and then re-started for every next launch. This leads to a fluctuating gap between subsequent repetitions. But it allows to wait for a start trigger every time. 

Another difference - **sequence duration requirement**. In-stream looping can only be used for a sufficiently long sequence - single-rep duration should be longer than `chunksize` (typically 150 ms). Repetitive re-launching does not have such a limitation and can be used with a sequence of any duration.

Both mechanisms allow for mid-run interruption with `KeyboardInterrupt` - streamer will stop after completing the current repetition in progress. 

A more advanced example - a minimal custom "progress bar" to monitor runs with a large number of in-stream repetitions. 

- We are using `wait_until_finished` method with a non-`None` argument - it blocks until eithr generation is finished or timeout elapses (see docstring)
- That way we can periodically poll `reps_written_count()` - the total number of repetitions written so far

In [8]:
instream_reps = 100

with ni_strmr.init_stream() as stream:
    stream.launch(instream_reps=instream_reps)
    
    while True:
        finished = stream.wait_until_finished(timeout=1)
        print(f'{stream.reps_written_count()} reps written out of {instream_reps}', end='\r')
        if finished:
            break

Below is the full list of `StreamHandle` methods:

* `launch`

* `wait_until_finished`

* `reps_written_count` - returns the total number of in-stream reps _written_ already (not the same as _generated_ already, see docstring)

* `request_stop` (you don't need this method in most cases)

You can find more info in the corresponding docstrings.